# Create the Topography of the 79NG Fjord GETM Setup

This notebook creates the topography of the 79° North Glacier (79NG) fjord in a format that is suitable for GETM.
The topography consists of the bathymetry, extracted from the RTopo dataset, and the ice thickness, extracted from the BedMachine dataset.
Furthermore, the indices of the grid points that form the grounding line are determined.
The resolution of the topography product is about 500 m in latitude and longitude ($\Delta \phi = 1/240° \approx 0.00417°$, $\Delta \lambda = 3/120° = 0.025°$).

To use this notebook:
 1. Download the file `RTopo-2.0.4_30sec_Greenland.nc` from the [dataset by Schaffer et al. (2019)](https://doi.org/10.1594/PANGAEA.905295) into a folder `data`.
 2. Download the file `BedMachineGreenland-v5.nc` from the [dataset by Morlighem et al. (2022)](https://doi.org/10.5067/GMEVBWFLWA7X) into a folder `data`.

**Acknowledgement:**
Many thanks to Samira Zander for valuable preparatory work during an internship at IOW in 2021.

**References:**
 * Haney, R. (1991): _On the Pressure Gradient Force over Steep Topography in Sigma Coordinate Ocean Models,_ Journal of Physical Oceanography, https://doi.org/10.1175/1520-0485(1991)021%3C0610:OTPGFO%3E2.0.CO;2
 * Klingbeil, K. et al. (2018): _The numerics of hydrostatic structured-grid coastal ocean models: State of the art and future perspectives,_ Ocean Modelling, https://doi.org/10.1016/j.ocemod.2018.01.007
 * Morlighem, M.  et al. (2022): _IceBridge BedMachine Greenland, Version 5,_ NASA National Snow and Ice Data Center Distributed Active Archive Center, https://doi.org/10.5067/GMEVBWFLWA7X (dataset)
 * Reinert, M. et al. (2023): _High-Resolution Simulations of the Plume Dynamics in an Idealized 79°N Glacier Cavity Using Adaptive Vertical Coordinates,_ Journal of Advances in Modeling Earth Systems, https://doi.org/10.1029/2023MS003721
 * Schaffer, J. et al. (2019): _An update to Greenland and Antarctic ice sheet topography, cavity geometry, and global bathymetry (RTopo-2.0.4),_ PANGAEA, https://doi.org/10.1594/PANGAEA.905295 (dataset)

Notebook by Markus Reinert (IOW, 2023–2024, https://orcid.org/0000-0002-3761-8029).

In [ ]:
import gsw
import numpy as np
import xarray as xr
import cmocean
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pyproj import CRS, Transformer

from tools.smoothing import smooth_2D_array
from tools.configuration import Configuration

In [ ]:
config = Configuration()

## Prepare the coordinate transformation

### CRS of RTopo

The RTopo dataset is in lat–lon coordinates, which we also want to use for the GETM setup.
RTopo has a resolution of $\Delta \phi = \Delta \lambda = 1/120°$, which is highely anisotropic at 79°N, i.e., the distance in meters is much larger in meridional than in longitudinal direction: $\Delta y \gg \Delta x$.
To obtain an approximately squared model grid ($\Delta x \approx \Delta y$), we have to choose a coarser resolution in longitudinal than in meridional direction: $\Delta \lambda > \Delta \phi$.
The anisotropy also has the effect that lat–lon coordinates are not feasible for interpolation.

In [ ]:
crs_latlon = CRS.from_epsg(4326)
crs_latlon

### CRS of BedMachine

The BedMachine dataset is in Cartesian coordinates.
Since distances in this coordinate system are close to physical distances on Earth, we can use these coordinates for interpolation.

In [ ]:
crs_cartesian = CRS.from_epsg(3413)
crs_cartesian

### CRS transformation

In [ ]:
transform_latlon_to_xy = Transformer.from_crs(crs_latlon, crs_cartesian).transform
transform_xy_to_latlon = Transformer.from_crs(crs_cartesian, crs_latlon).transform

## Load the RTopo dataset

In [ ]:
rtopo = xr.open_dataset("data/RTopo-2.0.4_30sec_Greenland.nc")

# Correct the names of the dimensions ("lat"/"lon" instead of "latdim"/"londim")
rtopo = rtopo.set_index(latdim="lat", londim="lon")
rtopo = rtopo.rename({"latdim": "lat", "londim": "lon"})

# Choose an area around 79NG that is a bit larger than the desired model domain
rtopo = rtopo.sel(lon=slice(-23, -14), lat=slice(79.1, 80.4))

# Check that the mask of the floating ice tongue (amask = 2) is simply given by all
# points at which the ice draft is below the sea level but above the sea floor.
# This will allow us later to re-construct the mask from regridded data.
assert np.all(
    ((rtopo.ice_base_topography > rtopo.bedrock_topography) & (rtopo.ice_base_topography < 0))
    == (rtopo.amask == 2)
), "mask of floating ice tongue cannot be reconstructed as expected"

# Compute and add Cartesian coordinates
X, Y = transform_latlon_to_xy(*xr.broadcast(rtopo.lat, rtopo.lon))
rtopo.coords["x"] = (["lat", "lon"], X)
rtopo.coords["y"] = (["lat", "lon"], Y)
rtopo.x.attrs = {"long_name": crs_cartesian.axis_info[0].name, "units": "m", "CRS": str(crs_cartesian)}
rtopo.y.attrs = {"long_name": crs_cartesian.axis_info[1].name, "units": "m", "CRS": str(crs_cartesian)}

rtopo

### Compute the grid resolution

In [ ]:
# Compute grid resolution in degrees
d_lat = np.diff(rtopo.lat)
d_lon = np.diff(rtopo.lon)

# Compute grid resolution in meters
LON, LAT = np.meshgrid(rtopo.lon, rtopo.lat)
dist_lat = gsw.distance(LON, LAT, axis=0)
dist_lon = gsw.distance(LON, LAT, axis=1)

print(f"Meridional   resolution: {d_lat.min():.6f}° to {d_lat.max():.6f}° ({dist_lat.min():.2f} m to {dist_lat.max():.2f} m)")
print(f"Longitudinal resolution: {d_lon.min():.6f}° to {d_lon.max():.6f}° ({dist_lon.min():.2f} m to {dist_lon.max():.2f} m)")

### Show the data

In [ ]:
fig, axs = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, dpi=300, figsize=(10, 3))
fig2, axs2 = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, dpi=300, figsize=(10, 2.4))

fig.suptitle(f"Ellipsoidal coordinates ({crs_latlon})")
fig2.suptitle(f"Cartesian coordinates ({crs_cartesian})")

for ax, ax2, var, cmap in zip(
    axs, axs2, ["bedrock_topography", "ice_base_topography", "amask"], [cmocean.cm.topo, None, None]
):
    im = rtopo[var].plot(ax=ax, cmap=cmap)
    im2 = rtopo[var].plot(ax=ax2, x="x", y="y", cmap=cmap)
    ax.set_title(var)
    ax2.set_title(var)
    ax2.set_aspect("equal")
    if var == "amask":
        im.colorbar.set_ticks(np.arange(4))
        im2.colorbar.set_ticks(np.arange(4))
for ax in [*axs[1:], *axs2[1:]]:
    ax.set_ylabel("")

## Load the BedMachine dataset

In [ ]:
bedmachine = xr.open_dataset("data/BedMachineGreenland-v5.nc")

# Check that the expected coordinate reference system is used
assert str(crs_cartesian).lower() in bedmachine.proj4, f"BedMachine seems to use a coordinate system other than {crs_cartesian}"
# Invert the y-axis to have increasing coordinates
bedmachine = bedmachine.isel(y=slice(None, None, -1))
# Crop to the same extent as RTopo
bedmachine = bedmachine.sel(x=slice(rtopo.x.min(), rtopo.x.max()), y=slice(rtopo.y.min(), rtopo.y.max()))

# Compute and add ellipsoidal coordinates
LAT, LON = transform_xy_to_latlon(*xr.broadcast(bedmachine.x, bedmachine.y))
bedmachine.coords["lat"] = (["x", "y"], LAT)
bedmachine.coords["lon"] = (["x", "y"], LON)
bedmachine.lat.attrs = {"long_name": crs_latlon.axis_info[0].name, "units": "degree", "CRS": str(crs_latlon)}
bedmachine.lon.attrs = {"long_name": crs_latlon.axis_info[1].name, "units": "degree", "CRS": str(crs_latlon)}

bedmachine

### Compute the grid resolution

In [ ]:
dx = np.diff(bedmachine.x)
dy = np.diff(bedmachine.y)

assert all(dx == dx[0]), "x-resolution is not constant"
assert all(dy == dy[0]), "y-resolution is not constant"
assert dx[0] == dy[0], "x- and y-resolutions differ"

print(f"Resolution in x- and in y-direction: {dx[0]} {bedmachine.x.units}")

### Show the data

In [ ]:
fig, axs = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, dpi=300, figsize=(10, 2.5))
fig2, axs2 = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, dpi=300, figsize=(10, 2.5))

fig.suptitle(f"Cartesian coordinates ({crs_cartesian})")
fig2.suptitle(f"Ellipsoidal coordinates ({crs_latlon})")

for ax, ax2, var, cmap in zip(axs, axs2, ["bed", "thickness", "mask"], [cmocean.cm.topo, None, None]):
    im = bedmachine[var].plot(ax=ax, cmap=cmap)
    im2 = bedmachine[var].plot(ax=ax2, x="lon", y="lat", cmap=cmap)
    ax.set_title(var)
    ax2.set_title(var)
    ax.set_aspect("equal")
    if var == "mask":
        im.colorbar.set_ticks(np.arange(4))
        im2.colorbar.set_ticks(np.arange(4))
for ax in [*axs[1:], *axs2[1:]]:
    ax.set_ylabel("")

## Show the alignment between the two datasets

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4), dpi=300, constrained_layout=True)
for mask, name, ice_value, cmap in [
    (bedmachine.mask, "BedMachine", 3, "Reds"), (rtopo.amask, "RTopo", 2, "Blues")
]:
    im = mask.where((mask == 0) | (mask == ice_value)).plot(
        x="x", y="y", vmin=-2 * ice_value, vmax=2 * ice_value, cmap=cmap, alpha=0.5
    )
    im.colorbar.set_label(name)
    im.colorbar.set_ticks([-2 * ice_value, 0, ice_value])
    im.colorbar.set_ticklabels(["", "", ""] if name == "RTopo" else ["land", "ocean", "ice tongue"])
ax.set_title("Comparison of the masks of the two datasets")
ax.set_aspect("equal")

## Create the topography dataset

To transform the RTopo grid into a grid with 500 m resolution, we cut the latitude grid in halves and reduce the longitude grid by combining every three grid cells into one.
This yields a meridional resolution of 1/240°, which is close to 500 m, and a longitudinal resolution of 3/120° = 0.025°, which is also close to 500 m (shorter than 500 m in the North of the domain, longer than 500 m in the South).

In [ ]:
n_divide_lat = 2
n_filter_lon = 3

In [ ]:
ds = xr.Dataset(
    {"grid_type": 2, "dlat": 1 / 120 / n_divide_lat, "dlon": n_filter_lon / 120},
    attrs={
        "title": "Topography of the 79NG fjord for GETM",
        "author": "Markus Reinert (ORCID: 0000-0002-3761-8029)",
        "institution": "Leibniz Institute for Baltic Sea Research Warnemuende (IOW), Germany",
        "description": "Bathymetry and ice thickness of the 79NG fjord in Greenland "
                       "on a regular lat-lon grid with a resolution of approximately 500 m, "
                       "to be used with the regional ocean model GETM.",
        "comment": "When opening this dataset with ncview, use the option `-minmax all` "
                   "to see the full range of the topography.",
    },
)

# Add coordinates
ds.coords["lat"] = np.arange(79.1, 80.401, ds.dlat)
ds.coords["lon"] = np.arange(-23., -13.99, ds.dlon)

# Add attributes
ds.lat.attrs = {"long_name": "latitude" , "units": "degree", "CRS": str(crs_latlon)}
ds.lon.attrs = {"long_name": "longitude", "units": "degree", "CRS": str(crs_latlon)}
ds.dlat.attrs = {"long_name":   "meridional resolution", "units": "degree"}
ds.dlon.attrs = {"long_name": "longitudinal resolution", "units": "degree"}
ds.grid_type.attrs = {"description": "equi-distant spherical grid"}

# Check for consistency with RTopo
assert np.allclose(ds.lat[::n_divide_lat], rtopo.lat), "Latitude does not match RTopo"
assert np.allclose(ds.lon, rtopo.lon[::n_filter_lon]), "Longitude does not match RTopo"

ds

### Check the grid resolution

In [ ]:
# Compute grid resolution in meter
LON, LAT = np.meshgrid(ds.lon, ds.lat)
dist_lat = gsw.distance(LON, LAT, axis=0)
dist_lon = gsw.distance(LON, LAT, axis=1)

assert np.allclose(ds.dlat, np.diff(ds.lat)), "meridional distance is not correct"
assert np.allclose(ds.dlon, np.diff(ds.lon)), "longitudinal distance is not correct"
assert np.allclose(dist_lat, dist_lat[0, 0]), "meridional distance in meter is not constant"
assert not np.allclose(dist_lon, dist_lon[0, 0]), "longitudinal distance in meter is almost constant"

print(f"{ds.dlat.long_name}:   {ds.dlat:.5f}° ({dist_lat[0, 0]:.2f} m)")
print(f"{ds.dlon.long_name}: {ds.dlon:.5f}° ({dist_lon.min():.2f} m to {dist_lon.max():.2f} m)")

### Add Cartesian coordinates

In [ ]:
X, Y = transform_latlon_to_xy(*xr.broadcast(ds.lat, ds.lon))
ds.coords["x"] = (["lat", "lon"], X)
ds.coords["y"] = (["lat", "lon"], Y)
ds.x.attrs = {"long_name": crs_cartesian.axis_info[0].name, "units": "m", "CRS": str(crs_cartesian)}
ds.y.attrs = {"long_name": crs_cartesian.axis_info[1].name, "units": "m", "CRS": str(crs_cartesian)}

### Reduce the longitudinal resolution of RTopo

If the library `bottleneck` is installed, we need to deactivate it for the computation of the moving average, because of a strange bug in the mean-computation: https://github.com/pydata/xarray/issues/1346.
This wrong computation of the mean causes ice base topography and bed rock topography to differ slightly after the averaging, so we selection of the mask gives a wrong result.

In [ ]:
# Why does the selection work without method="nearest"?
# Maybe because rtopo.lon is float32, but ds.lon is float64?
with xr.set_options(use_bottleneck=False):
    ice_base = rtopo.ice_base_topography.rolling(lon=n_filter_lon, center=True).mean().sel(lon=ds.lon)
    bed_rock = rtopo.bedrock_topography .rolling(lon=n_filter_lon, center=True).mean().sel(lon=ds.lon)

### Interpolate RTopo in meridional direction

As the grids of RTopo and the topography dataset are not exactly equal, the interpolation results in slightly different values at the grid locations that already exist in RTopo.
However, this difference is less than 0.6 m and much smaller in most places, so not relevant.

In [ ]:
ice_base = ice_base.interp(lat=ds.lat)
bed_rock = bed_rock.interp(lat=ds.lat)

### Smooth the RTopo data

We smooth the topography using a running mean of size 3×3 to obtain a simpler mask.

In [ ]:
ice_base_smooth = smooth_2D_array(ice_base, 1, 1)
bed_rock_smooth = smooth_2D_array(bed_rock, 1, 1)

### Create the mask

#### Compute the ocean mask

We take as water/ocean points all grid cells where
 * the bedrock is below sea level **and**
 * the ice base is above the sea floor.

According to the check we made when loading RTopo, this fits with the RTopo mask.

We remove points where the ocean is only 2 m shallow or less, since these grid cells might fall dry during ebb.
Masking these points gives a clearly simpler domain boundary and reduces problems with misaligning ice/ocean data.

In [ ]:
ds["mask"] = (
    ("lat", "lon"),
    (bed_rock_smooth < -2) & (ice_base_smooth > bed_rock_smooth),
    {
        "long_name": "ocean mask",
        "source": "RTopo-2.0.4 (Schaffer et al. 2019)",
        "description": "The ocean mask is True on all water points "
                       "and False on points with land or grounded ice.",
    },
)
cbar_kwargs = {"ticks": [0, 1], "format": lambda x, pos: "ocean" if x else "land", "label": ""}
ds.mask.plot(cbar_kwargs=cbar_kwargs)
ds.mask

#### Remove isolated lakes

Isolated lakes with no connection to the ocean are irrelevant for the model, so we set them to land to simplify the shape of the model domain.

In [ ]:
mask_lake = ds.mask.sel(lat=slice(80.299, 80.315), lon=slice(-16.38, -16.34))
print(f"Selection in the North contains {mask_lake.size} points of which {np.count_nonzero(mask_lake)} are water points that are put to land now.")
mask_lake[...] = False

mask_lake = ds.mask.sel(lat=80.0875, lon=-20.3, method="nearest")
print(f"Selection 1 near Dijmphna Sound contains 1 {'water point that is put to land now' if mask_lake else 'land point'}.")
mask_lake[...] = False

mask_lake = ds.mask.sel(lat=80.096, lon=-20.375, method="nearest")
print(f"Selection 2 near Dijmphna Sound contains 1 {'water point that is put to land now' if mask_lake else 'land point'}.")
mask_lake[...] = False

ds.mask.plot(cbar_kwargs=cbar_kwargs);

#### Compare the created ocean mask with the original masks

In [ ]:
fig, ax = plt.subplots(constrained_layout=True, dpi=300)
ds.mask.plot(cmap="Greys_r", alpha=0.5, cbar_kwargs=cbar_kwargs)
for name, mask, ice_value in [("BedMachine", bedmachine.mask, 3), ("RTopo", rtopo.amask, 2)]:
    cs = ((mask == 0) | (mask == ice_value)).plot.contour(
        x="lon", y="lat", levels=[0.5], linewidths=0.5, colors=name[0].lower()
    )
    plt.plot([], [], color=cs.get_edgecolor(), lw=cs.get_linewidth(), label=name)
plt.legend(loc="upper left", title="Mask of dataset")
ax.set_title("Comparison of the created mask with the original masks")
ax.set_xlim(-22.8, -15.5)
ax.set_aspect(ds.dlon / ds.dlat)

In the vicinity of the grounding line, the created mask (grey and white areas in the background) has more ocean points than the masks of the two original datasets (red and blue lines).
We will adjust the mask there manually to obtain a better fit with the data.

#### Bring the BedMachine mask to the model grid

In [ ]:
# Make the BedMachine mask binary: False for land, True for ocean
bm_mask = (bedmachine.mask == 0) | (bedmachine.mask == 3)
# Get rid of latitude and longitude to prevent a clash of dimensions when interpolating
del bm_mask["lat"], bm_mask["lon"]
# Interpolate the BedMachine mask to the model grid
bm_mask = bm_mask.astype(float).interp(x=ds.x, y=ds.y)
# Make the array Boolean (again) by applying a 90%-threshold
bm_mask = bm_mask > 0.9

#### Adjust the mask in the southwestern corner of the fjord

In the area shown in the following figure, we adjust the mask manually to be more consistent with the datasets.
Without adjustment, the model showed high barotropic velocities in this area.
Our adjustment consists of two steps:
* We mask the southernmost row of ocean points in the fjord, since they are land (or grounded ice) in both datasets.
* We use the BedMachine mask for the southeastern corner of the fjord, since the RTopo mask looks irregular there.

In [ ]:
fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 3), dpi=300)

ds.mask.plot(cmap="Greys_r", alpha=0.5, cbar_kwargs=cbar_kwargs | {"label": "Created mask"})
ds.mask.plot.contour(levels=[0.5], colors="k")
plt.plot([], [], "k", label="Created mask")
bm_mask.plot(cmap="Blues", alpha=0.5, cbar_kwargs={"ticks": [], "label": "BedMachine mask"})

for mask, name, ice_value in [(bedmachine.mask, "BedMachine", 3), (rtopo.amask, "RTopo", 2)]:
    cs = ((mask == 0) | (mask == ice_value)).plot.contour(x="lon", y="lat", levels=[0.5], colors=name[0].lower())
    plt.plot([], [], color=cs.get_edgecolor(), label=name)
plt.legend(loc="lower right", fontsize="small")

ax.set_title("Zoom to the grounding line South")
# Show the model grid as squares with lables in cell centers
ax.set_xticks(ds.lon[::10])
ax.set_yticks(ds.lat[::6])
ax.set_xticks(ds.lon - ds.dlon/2, minor=True)
ax.set_yticks(ds.lat - ds.dlat/2, minor=True)
ax.set_xlim(-22.55 - ds.dlon/2, -21.75 + ds.dlon/2)
ax.set_ylim(79.25 - ds.dlat/2, 79.35 + ds.dlat/2)
ax.set_aspect(ds.dlon / ds.dlat)
ax.grid(which="minor")

In [ ]:
# Mask the southernmost row of ocean points in the fjord
lat_fjord_S = ds.lat.sel(lat=79.25 + 4 * ds.dlat, method="nearest")
print(f"Selected row at {lat_fjord_S:.4f}°N.")
assert np.all(ds.mask.sel(lon=slice(-21), lat=slice(lat_fjord_S - ds.dlat/2)) == False), "selected latitude is not southern most ocean row in the fjord"
mask_fjord_S = ds.mask.sel(lon=slice(-21), lat=lat_fjord_S)
print("Contains", np.count_nonzero(mask_fjord_S), "ocean points in the fjord that are put to land now.")
mask_fjord_S[:] = False

In [ ]:
# Use the BedMachine mask for the southeastern corner of the fjord
ds["mask"] &= bm_mask.where((ds.lon >= -22.35) & (ds.lon <= -21.8) & (ds.lat < 79.35), True)
ds.mask.plot(cbar_kwargs=cbar_kwargs);

### Add the bathymetry

In [ ]:
ds["bathymetry"] = (
    ("lat", "lon"),
    -bed_rock_smooth,
    {
        "long_name": "bottom depth",
        "units": "m",
        "positive": "down",
        "source": "RTopo-2.0.4 (Schaffer et al. 2019)",
    },
)
ds["bathymetry"] = ds.bathymetry.where(ds.mask)
# Smooth the bathymetry with a 5×5 running mean filter
ds.bathymetry.data = smooth_2D_array(ds.bathymetry, 2, 2)
ds.bathymetry.plot(cmap=cmocean.cm.deep)
plt.title("Bathymetry");

### Process the ice thickness

#### Interpolate the data to the model grid

For the interpolation, Cartesian coordinates must be used.
We then apply the mask of the model grid by multiplication, so that we have 0 (and not NaN) on land.

In [ ]:
# Get rid of latitude and longitude to prevent a clash of dimensions when interpolating
bm_thickness = bedmachine.thickness
del bm_thickness["lat"], bm_thickness["lon"]
ds["glIceD"] = bm_thickness.interp(x=ds.x, y=ds.y) * ds.mask
ds.glIceD.attrs = {
    "long_name": "glacial ice thickness",
    "units": "m",
    "source": "BedMachine v5.5 (Morlighem et al. 2022)",
}

plt.figure(dpi=300)
ds.mask.plot(cbar_kwargs=cbar_kwargs, vmax=2, cmap="Reds", alpha=1/2)
ds.glIceD.where(ds.glIceD).plot(cmap=cmocean.cm.ice)
plt.title("Ice thickness");

#### Remove isolated ice columns

Along the coasts of Dijmphna Sound and Hovgaard Island, there are a few isolated ice columns.
Since the original data does not show any isolated ice columns, they can be considered artificats due to the misalignment of the masks.
Given that the 79NG ice tongue lies below 79.83°N, we remove all ice further north.

In [ ]:
ice_north = ds.glIceD.sel(lat=slice(79.83, 80.4))
print(f"Selection contains {np.count_nonzero(ice_north)} ice columns, which are removed now.")
ice_north[...] = 0

#### Smooth the ice thickness

Similar to the smoothing of the bathymetry, we use a running mean of size 3×3.

Note:
We smooth the ice also between ice and ocean (but not between ice and land), resulting in a larger ice tongue after the smoothing than before.
This reduces the slopes at the calving front and fills out ice-free points in the fjord that come from the mismatch of the masks between RTopo and BedMachine.

In [ ]:
ice_smooth = smooth_2D_array(ds.glIceD.where(ds.mask), 1, 1)
ds.glIceD.data = np.where(np.isfinite(ice_smooth), ice_smooth, 0)

plt.figure(dpi=300)
ds.mask.plot(cbar_kwargs=cbar_kwargs, vmax=2, cmap="Reds", alpha=1/2)
ds.glIceD.where(ds.glIceD).plot(cmap=cmocean.cm.ice)
plt.title("Ice thickness");

### Crop to the model domain

The chosen extent is 4 cells larger than the domain of interest, so that the open boundaries and the sponge zones (3 grid points adjacent to each open boundary) are *not* within the domain of interest.
In the West, there is no open boundary; we keep one row of land points to make it clear that this boundary is closed.

In [ ]:
# Get the index of the westernmost ocean point
i_ocean_west = np.where(ds.mask)[1].min()
lon_min = ds.lon[i_ocean_west - 1]
lon_max = -15. + 4.5 * ds.dlon
lat_min = 79.2 - 4.5 * ds.dlat
lat_max = 80.3 + 4.5 * ds.dlat
ds = ds.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))

print(f"Topography has {ds.bathymetry.size} grid points")
print(f"of which {np.count_nonzero(ds.mask) / ds.bathymetry.size * 100:2.0f} %",
      f"({np.count_nonzero(ds.mask):5}) are water")
print(f"     and {np.count_nonzero(~ds.mask) / ds.bathymetry.size * 100:2.0f} %",
      f"({np.count_nonzero(~ds.mask):5}) are land.")

ds

In [ ]:
lon_size = int(config.get_text("getm/domain/ih"))
lat_size = int(config.get_text("getm/domain/jh"))
assert lon_size == ds.lon.size, "grid size in longitude does not match configuration"
assert lat_size == ds.lat.size, "grid size in latitude does not match configuration"

### Simplify the sponge zone

In the sponge zone (open boundary points and 3 adjacent grid points), we make the topography flat and the coastline straight to reduce problems near the open boundaries.

In [ ]:
ds.bathymetry[:4] = ds.bathymetry[4]
ds.bathymetry[-5:] = ds.bathymetry[-5]
ds.bathymetry[:, -5:] = ds.bathymetry[:, -5]

# Update the mask to fit the adjusted topography
ds.mask.data = np.isfinite(ds.bathymetry)

### Prevent the ice from hitting the ground

In the created dataset, there are a few grid points near the grounding line and along the coastline of the fjord at which the ice would be grounded.
(We consider those places as grounded where the water column is less than 2 m thick in the initial state.)
In these locations, both the bathymetry from RTopo and the ice topography from BedMachine are – to some extent – a product of interpolation and not directly of measurements, so the data should be taken with a grain of salt.
We prevent the ice from actually hitting the ground in the model by getting rid of grounded ice in three ways:
* Away from the grounding line, grounded ice occurs only along the fjord walls and we simply set these locations to land.
* In the northern corner of the grounding line, the ice appears to be partly grounded.  A closer look at RTopo puts the reliability of the data in this area in question.  The water column is about 1.1 m thick there.  Having a constant water column thickness over an extended area could indicate that the data is aritifical.  Therefore, we put this entire corner to land.  This includes 19 grid points more than those where the ice is grounded.
* In the vicinity of the grounding line, grounded ice is more widespread.  Since our main interest lies in processes at the ice–ocean interface, we use the ice topography as is, assuming that the data in BedMachine is correct.  We adjust the bottom topography as needed for the ice to float.

To determine if ice is floating or grounded (see Reinert et al., 2023, Section 2.2.1), we use the reference density $\rho_0$ instead of the actual sea water density $\rho$, which is a function of space and time.
The difference between these two densities is generally less than 1 %.
The reference density is also used by GETM to initialize the ice in its (approximate) vertical equilibrium position.

In [ ]:
rho_0 = float(config.get_text("getm/parameters/rho_0"))
rho_ice = float(config.get_text("getm/ice/rho_ice"))

print(f"The reference density is {rho_0 = :6.1f} kg/m³,")
print(f"glacial ice density is {rho_ice = :6.1f} kg/m³ ({rho_ice/rho_0*100:.1f} % of rho_0).")

In [ ]:
min_WCT = 2
ice_draft = ds.glIceD * rho_ice / rho_0
is_grounded = ds.bathymetry < ice_draft + min_WCT

plt.figure(dpi=600)
is_grounded.where(ds.mask).plot(cmap="Reds")
plt.title("Points with originally grounded ice (dark red)")
print(f"There are {np.count_nonzero(is_grounded)} grid points with grounded ice.")

In [ ]:
# Away from the grounding line, put grid points with grounded ice to land
is_grounded_far = is_grounded.where((ds.lon > -22) | (ds.lat > 79.49), False)
print(f"Away from the grounding line, ice is grounded on {np.count_nonzero(is_grounded_far)} grid points that are put to land now.")
ds["mask"] &= ~is_grounded_far

In [ ]:
# Make the mask in the northern corner of the grounding line more regular
mask_gline_N = ds.mask.sel(lat=slice(79.5 - 5/2 * ds.dlat, None), lon=slice(-22.5 - ds.dlon))
print(f"Selection contains {np.count_nonzero(mask_gline_N)} water points that are put to land now.")
mask_gline_N[...] = False

mask_gline_N = ds.mask.sel(lat=slice(79.5 - 3/2 * ds.dlat, None), lon=slice(-22.4))
print(f"Selection contains {np.count_nonzero(mask_gline_N)} water points that are put to land now.")
mask_gline_N[...] = False

In [ ]:
print(f"The bathymetry is adjusted in {np.count_nonzero(is_grounded & ds.mask)} remaining points with grounded ice.")
ds.bathymetry.data = np.where(is_grounded, ice_draft + min_WCT, ds.bathymetry)

In [ ]:
# Apply the new mask
ds["bathymetry"] = ds.bathymetry.where(ds.mask)
ds["glIceD"] = ds.glIceD.where(ds.mask, 0)
ice_draft = ds.glIceD * rho_ice / rho_0

### Analyse the local slopes of the topography

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, sharey=True, constrained_layout=True, figsize=(10, 7), dpi=300)
for axs_row, dim in zip(axs, ["lat", "lon"]):
    for ax, var in zip(axs_row, ["bathymetry", "glIceD"]):
        ds[var].where(ds.mask).diff(dim).plot(ax=ax)
        ax.set_title(f"Slope of {var} in {dim}-direction")

### Check hydrostatic consistency

Haney (1991) proposed a condition to check for hydrostatic consistency in $\sigma$-coordinate models.
Our GETM setup uses adaptive vertical coordinates, which greatly reduce problems with the internal pressure gradient, in comparison to $\sigma$-coordinates.
Nevertheless, the Haney condition can give us a first idea at which locations problems might occur.
We apply the check for an equidistant distribution of $K = 100$ vertical levels, which is the layer distribution at the beginning of the model run.

According to Equation 1 by Haney (1991) and Chapter 7.2 by Klingbeil et al. (2018), the number
$$r = K \frac{|\Delta D|}D$$
should be kept at a reasonable level, where $D$ is the water column thickness and $\Delta D$ is its difference in zonal/meridional direction.
In the 2D model of the 79NG fjord by Reinert et al. (2023), the maximum of $r$ is around 70 near the grounding line, so we aim to keep the value of $r$ below 100.

In [ ]:
K = 100
D = ds.bathymetry - ice_draft

fig, ax_grid = plt.subplots(3, 2, sharex=True, sharey=True, constrained_layout=True, figsize=(9, 10), dpi=300)
for axs, dim in zip(ax_grid.T, ["lat", "lon"]):
    r = K * np.abs(D.diff(dim)) / D
    r.plot(ax=axs[0], cmap=cmocean.cm.amp, vmax=1e2, add_colorbar=dim == "lon")
    r.plot(ax=axs[1], cmap=cmocean.cm.balance, norm=LogNorm(1e-2, 1e2), add_colorbar=dim == "lon")
    r.plot(ax=axs[2], cmap=cmocean.cm.balance, vmax=20, add_colorbar=dim == "lon")
    axs[0].set_title(f"$r$-value in {dim}-direction")
for ax in ax_grid[:-1].flatten():
    ax.set_xlabel("")
for ax in ax_grid[:, 1:].flatten():
    ax.set_ylabel("")
for ax in ax_grid.flatten():
    ax.set_aspect(ds.dlon / ds.dlat)

### Determine the grounding line

We consider as the grounding line the westernmost grid points of the water domain south of 79.49°N at 22.4°W and to the West of it.

In [ ]:
gline = (-1 * ds.mask).argmin("lon").sel(lat=slice(79.49))
gline = gline.where(gline <= abs(ds.lon + 22.4).argmin(), other=-1, drop=True)
print(f"The grounding line consists of {gline.size} grid points.")

### Show the topography

#### Compute the coordinates of the grounding line

We show the grounding line in a step-wise manner along the western boundary of the grid cells that belong to the grounding line.

In [ ]:
gline_lat = []
gline_lon = []
for lon, lat in zip(ds.lon.isel(lon=gline), gline.lat):
    gline_lat += [lat - ds.dlat / 2, lat + ds.dlat / 2]
    gline_lon += [lon - ds.dlon / 2, lon - ds.dlon / 2]

#### Create the figure

In [ ]:
fig, axs = plt.subplots(ncols=3, sharex=True, sharey=True, constrained_layout=True, figsize=(18, 4.5), dpi=300)

ax = axs[0]
ds.bathymetry.plot(ax=ax, cmap=cmocean.cm.deep)
ax.set_title(f"Bathymetry around 79NG\nfrom {ds.bathymetry.source}")
ax.plot(gline_lon, gline_lat, "r", lw=1, solid_capstyle="butt", label="grounding line")
ax.legend(loc="lower left", borderaxespad=0.1)

ax = axs[1]
ds.glIceD.where(ds.glIceD).plot(ax=ax, cmap=cmocean.cm.ice_r)
ax.set_title(f"Ice thickness of the 79NG ice tongue\nfrom {ds.glIceD.source}")
ax.set_ylabel("")

ax = axs[2]
im = (ds.bathymetry - ice_draft).plot(ax=ax, cmap=cmocean.cm.amp)
im.colorbar.set_label("Water column thickness [m]")
ax.set_title(f"Water column thickness around 79NG\nfor $\\{rho_0 = :.0f}$ kg/m³ and $\\rho_{{\\text{{ice}}}} = {rho_ice:.0f}$ kg/m³")
ax.set_ylabel("")

# Set the aspect ratio such that grid cells are squares
for ax in axs:
    ax.set_aspect(ds.dlon / ds.dlat)

fig.savefig("figures/topography_500m.png")

### Construct $A_n$ as function of bathymetry

In [ ]:
if config.get_text("getm/m2d/An_method") != "2":
    print("An is not read from file, so no need to create An.")
else:
    An = float(config.get_text("getm/m2d/An_const"))
    ds["An"] = An * np.sqrt(ds.bathymetry / ds.bathymetry.max()).where(ds.mask, 0)
    ds.An.attrs = {
        "long_name": "horizontal numerical diffusion coefficient",
        "units": "m2/s",
        "description": f"square-root of bathymetry with maximum {An}",
    }
    ds.An.plot()

### Check the dataset

In [ ]:
assert np.all(ds.mask == np.isfinite(ds.bathymetry)), "bathymetry and mask do not match"
assert np.all(np.isfinite(ds.glIceD)), "glacial ice thickness contains invalid values"
assert np.all(ds.glIceD >= 0), "glacial ice thickness contains negative values"
ds

### Save the dataset

In [ ]:
filename = config.get_file_path("getm/rivers/river_info")
with open(filename, "w") as f:
    f.write("# Indices of the grid points along the 79NG grounding line\n")
    f.write("# on the GETM topography with 500 m resolution\n")
    f.write("# Subglacial runoff is uniformly distributed over these points.\n")
    f.write(f"{gline.size}\n")
    for ilon in gline:
        f.write(f"{ilon.data + 1} {np.where(ds.lat == ilon.lat)[0][0] + 1} runoff\n")
print(f"Saved the grounding line info as {filename!r}.")

In [ ]:
filename = config.get_file_path("getm/domain/bathymetry")
assert filename == config.get_file_path("getm/ice/ice_file"), "filenames of bathymetry and ice thickness are different"
assert filename == config.get_file_path("getm/m2d/An_file"), "filenames of bathymetry and An-file are different"
ds.to_netcdf(filename)
print(f"Saved the topography as {filename!r}.")